# Notebook to build a netcdf file for SMSpp starting by a pypsa model



This notebook executes the following procedure:
1. Create and optimize a pypsa model
2. Converts the PyPSA model into a InvestmentBlock problem to be read by SMSpp
3. Launches the optimization using the UCBlock solver of SMSpp
4. Validate the objective function of SMS++ and compares it to SMSpp

## 1. Creation of the PyPSA model

We create a simple PyPSA model arbitrary number of buses, few generators, storages and loads.

Main assumptions are:
- The network is purely radial in the form bus1 -> bus2 -> bus3 -> ... busN
- A load is added to each bus of the network
- The following technologies are supported:
  - fuel-fired diesel generator
  - pv generator
  - wind generator
  - battery
  - hydro unit

The following notebook, performs the following SMS++ to PyPSA conversion:

|Physical object|PyPSA object|SMS++ object|
|----------------|------------|------------|
|diesel generator|Generator|ThermalUnitBlock|
|pv generator|Generator|IntermittentUnitBlock|
|wind generator|Generator|IntermittentUnitBlock|
|battery|StorageUnit|BatteryUnitBlock|
|hydro unit|StorageUnit|HydroUnitBlock|

In [1]:
n_snapshots = 1*24 #365*24

buses_demand = [0] # list of buses where demand is located
bus_PV = 0 # Bus where PV is located; if none, no PV is considered
bus_wind = 0 # Bus where wind is located; if none, no wind is considered
bus_storage = None # Bus where storage is located; if none, no storage is considered
bus_hydro = None # Bus where hydro is located; if none, no hydro is considered
bus_diesel = 0 # Bus where diesel is located; if none, no pv is considered
add_load_shedding = True # when true, load shedding units are added

#### Preliminary imports

In [2]:
folder_builder = "../data/SMSpp/InvestmentBlockSolver"
path_smspp_pypsa = folder_builder + "/pypsa2smspp.nc4"

renewable_carriers = ["pv", "wind"]
thermal_carriers = ["diesel"]
slack_carriers = ["curtailment"]

In [3]:
import pypsa
from helpers import build_microgrid_model
import netCDF4 as nc
import pandas as pd
import numpy as np
import re

NC_DOUBLE = "f8"
NP_DOUBLE = np.float64
NC_UINT = "u4"
NP_UINT = np.uint32
NC_BYTE = "B"
NP_BYTE = np.byte

MAX_INFINITY = 1e6

#### The following code creates the desired pypsa model.

In [4]:
n = build_microgrid_model(
    n_snapshots = n_snapshots,
    buses_demand = buses_demand,
    bus_PV = bus_PV,
    bus_wind = bus_wind,
    bus_storage = bus_storage,
    bus_diesel = bus_diesel,
    bus_hydro=bus_hydro,
    x = 10.389754,
    y = 43.720810,
    hydro_factor=0.5,
    add_load_shedding=add_load_shedding,
    load_shedding=10000,
)

# n.storage_units.capital_cost /= 2
# n.storage_units.p_min_pu = 0.
# n.storage_units.p_nom_max = 10
# n.storage_units.max_hours = 0.5

# n.snapshot_weightings["stores"] = 1.0

# n.generators.p_nom = 100

In [5]:
n.optimize(solver_name="gurobi")

Index(['Bus 0'], dtype='object', name='Bus')
Index(['pv', 'wind', 'diesel', 'Curtailment_Bus 0'], dtype='object', name='Generator')
Index(['Bus 0'], dtype='object', name='Bus')
Index(['pv', 'wind', 'diesel', 'Curtailment_Bus 0'], dtype='object', name='Generator')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io: Writing time: 0.03s


Set parameter Username


INFO:gurobipy:Set parameter Username


Academic license - for non-commercial use only - expires 2025-07-18


INFO:gurobipy:Academic license - for non-commercial use only - expires 2025-07-18


...

Solved in 25 iterations and 0.03 seconds (0.00 work units)


INFO:gurobipy:Solved in 25 iterations and 0.03 seconds (0.00 work units)


Optimal objective  2.644249121e+04


INFO:gurobipy:Optimal objective  2.644249121e+04
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 99 primals, 219 duals
Objective: 2.64e+04
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper were not assigned to the network.


('ok', 'optimal')

#### Analyze the results

In [6]:
# Auxiliary function
def get_bus_idx(n, bus_series, dtype="uint32"):
    """
    Returns the numeric index of the bus in the network n for each element of the bus_series.
    """
    return bus_series.map(n.buses.index.get_loc).astype(dtype)

Calculate the marginal costs.

In [7]:
marg_cost = pd.concat(
    [
        n.generators_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.generators.marginal_cost),
        n.storage_units_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.storage_units.marginal_cost),
        n.links_t.p0.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.links.marginal_cost),
        n.stores_t.p.mul(n.snapshot_weightings.objective, axis=0).sum().mul(n.stores.marginal_cost),
    ],
)
marg_cost

pv                      0.000000
wind                    0.000000
diesel               4430.511599
Curtailment_Bus 0       0.000000
dtype: float64

Calculate the capital costs

In [8]:
cap_cost = pd.concat(
    [
        n.generators.eval("p_nom_opt * capital_cost"),
        n.storage_units.eval("p_nom_opt * capital_cost"),
        n.links.eval("p_nom_opt * capital_cost"),
        n.lines.eval("s_nom_opt * capital_cost"),
        n.stores.eval("e_nom_opt * capital_cost"),
    ]
)
cap_cost

pv                       0.000000
wind                 14638.517231
diesel                7373.462381
Curtailment_Bus 0        0.000000
dtype: float64

## 2. Convert the PyPSA model into a InvestmentBlock problem

The following code converts the pypsa model into a InvestmentBlock problem.
The conversions adopts the covnersion matrix adopted in the example above.

### 2.1 Create the SMS++ problem file

In [9]:
ds = nc.Dataset(path_smspp_pypsa, "w")


ds.setncattr("SMS++_file_type", 1)  # Set file type to 1 for problem file

### 2.2 Creates the Inner InvestmentBlock

#### Auxiliary functions needed to create the InvestmentBlock

In [10]:
obj_order = ["Generator", "StorageUnit", "Link", "Line", "Store"]
def sort_order(x):
    carrier_sort = {
        "diesel": 1,
        "pv": 2,
        "wind": 3,
        "battery": 4,
        "curtailment": 5,
        "hydro": 6,
    }
    if x in carrier_sort.keys():
        return carrier_sort[x]
    else:
        return max(carrier_sort.values()) + 1


# get the nominal name
def nom_obj(obj):
    if obj.lower() == "line":
        return "s_nom"
    elif obj.lower() == "store":
        return "e_nom"
    else:
        return "p_nom"

# get the extendable objects
dict_extendable = {
    obj: (
        n.df(obj)
        .sort_values(by=["carrier"], key=lambda x: x.map(lambda y: sort_order(y)))
        .reset_index()
        .rename(columns={obj: "name"})
        .query(f"{nom_obj(obj)}_extendable == True")
    )
    for obj in obj_order
}

# get list of component by type
def get_param_list(dict_objs, col, obj_order=obj_order, max_val=MAX_INFINITY):
    lvals = []
    initial_id = 0
    for obj in obj_order:
        if col == "type":
            lvals += [obj for i in range(len(dict_objs[obj].index))]
        elif col == "id": # get the id starting from the first object
            lvals += list(initial_id + dict_objs[obj].index)
        else:
            df_col = nom_obj(obj) + col[3:] if col.startswith("nom_") else col
            lvals += list(dict_objs[obj][df_col].values)
        initial_id += len(dict_objs[obj].index)
    if isinstance(lvals[0], float) or isinstance(lvals[0], int):
        lvals = [np.clip(val, -max_val, max_val) for val in lvals]
    return lvals

# Number of extendable assets
n_extendable = sum(len(df.index) for df in dict_extendable.values())

In [11]:
# Create UCBlock

inv_block = ds.createGroup("InvestmentBlock")  # Create the first main block

# master.id = "0"  # mandatory attribute for all blocks
inv_block.type = "InvestmentBlock"  # mandatory attribute for all blocks

# num of extendables
inv_block.createDimension("NumAssets", n_extendable)

# assets
assets = inv_block.createVariable("Assets", NC_UINT, ("NumAssets",))
assets[:] = get_param_list(dict_extendable, "id")

# investment cost
cost = inv_block.createVariable("Cost", NC_DOUBLE, ("NumAssets",))
cost[:] = get_param_list(dict_extendable, "capital_cost")

# Lower bound
lb = inv_block.createVariable("LowerBound", NC_DOUBLE, ("NumAssets",))
lb[:] = np.full((n_extendable,), 1e-6, dtype=NP_DOUBLE)
# lb[:] = get_param_list(dict_extendable, "nom_min")

# Upper bound
ub = inv_block.createVariable("UpperBound", NC_DOUBLE, ("NumAssets",))
ub[:] = get_param_list(dict_extendable, "nom_max")

# Installed Capacity
ic = inv_block.createVariable("InstalledCapacity", NC_DOUBLE, ("NumAssets",))
ic[:] = np.full((n_extendable,), 0., dtype=NP_DOUBLE)

# asset type
asset_type = inv_block.createVariable("AssetType", NC_BYTE, ("NumAssets",))
asset_type[:] = np.full((n_extendable,), 0, dtype=NP_BYTE)

### 2.3 Add the UCBlock as inner block of InvestmentBlock

In [12]:
# Add UCBlock
master = inv_block.createGroup("InnerBlock")  # Create the first main block

# master.id = "0"  # mandatory attribute for all blocks
master.type = "UCBlock"  # mandatory attribute for all blocks

Add the Network Data to the UCBlock (supported one node)

In [13]:
ndg = master.createGroup("NetworkData")
ndg.createDimension("NumberNodes", 1)

n_timesteps = len(n.snapshots)
master.createDimension("TimeHorizon", n_timesteps)  # Time horizon


n_units = len(n.generators) + len(n.storage_units) + len(n.stores)
master.createDimension("NumberUnits", n_units)  # Number of nodes
master.createDimension("NumberElectricalGenerators", n_units)  # Number of elec. units
# some unitblocks, do have multiple units inside [e.g. cascading hydro or CCGT], not considered now


n_nodes = len(n.buses)
master.createDimension("NumberNodes", n_nodes)  # Number of nodes

if len(n.lines) > 0:   
    master.createDimension("NumberLines", len(n.lines))  # Number of lines

# Lines

if len(n.lines) > 0:

    # generators' node
    all_generators = [bus_PV, bus_wind, bus_storage, bus_hydro, bus_diesel]
    all_generators = [x for x in all_generators if x is not None]
    if add_load_shedding:
        all_generators = add_load_shedding + list(range(1, len(n.buses)))  # add load shedding as a generator
    generator_node = master.createVariable("GeneratorNode", NC_UINT, ("NumberElectricalGenerators",))
    generator_node[:] = np.array(all_generators, dtype=NP_UINT)

    # start lines
    start_line = master.createVariable("StartLine", NC_UINT, ("NumberLines",))
    start_line[:] = get_bus_idx(n, n.lines.bus0).values
    # end lines
    end_line = master.createVariable("EndLine", NC_UINT, ("NumberLines",))
    end_line[:] = get_bus_idx(n, n.lines.bus1).values
    # Min power flow
    min_power_flow = master.createVariable("MinPowerFlow", NC_DOUBLE, ("NumberLines",))
    min_power_flow[:] = - n.lines.s_nom.values
    # Max power flow
    max_power_flow = master.createVariable("MaxPowerFlow", NC_DOUBLE, ("NumberLines",))
    max_power_flow[:] = n.lines.s_nom.values
    # Susceptance
    susceptance = master.createVariable("Susceptance", NC_DOUBLE, ("NumberLines",))
    susceptance[:] = 1 / n.lines.x.values


Add demand to the UCBlock

In [14]:
# demand by node
loads_t = n.loads_t.p_set
active_demand = master.createVariable("ActivePowerDemand", NC_DOUBLE, ("NumberNodes", "TimeHorizon",)) #("NumberNodes", "TimeHorizon"))
active_demand[:] = loads_t.values.transpose()  # indexing between python and SMSpp is different: transpose

### 2.4 Add the ThermalUnitBlock for each fuel-fired generator

In [15]:
id_thermal = 0

thermal_generators = n.generators[n.generators.index.isin(thermal_carriers)]

# min_power_pypsa = thermal_generators.eval("p_nom_opt * p_min_pu")
# max_power_pypsa = thermal_generators.eval("p_nom_opt * p_max_pu")
min_power_pypsa = thermal_generators.eval("p_min_pu")
max_power_pypsa = thermal_generators.eval("p_max_pu")
linear_term_pypsa = thermal_generators.marginal_cost

for (idx_name, row) in thermal_generators.iterrows():

    tub = master.createGroup(f"UnitBlock_{id_thermal}")
    tub.id = str(id_thermal)
    tub.type = "ThermalUnitBlock"

    # Create variables


    # MinPower
    min_power = tub.createVariable("MinPower", NC_DOUBLE, ("TimeHorizon",)) #, ("TimeHorizon",))
    min_power[:] = np.repeat(min_power_pypsa.loc[idx_name], n_timesteps)

    # MaxPower
    max_power = tub.createVariable("MaxPower", NC_DOUBLE, ("TimeHorizon",)) #, ("TimeHorizon",))
    max_power[:] = np.repeat(max_power_pypsa.loc[idx_name], n_timesteps)

    # StartUpCost
    start_up_cost = tub.createVariable("StartUpCost", NC_DOUBLE)
    start_up_cost[:] = 0.0

    # LinearTerm
    linear_term = tub.createVariable("LinearTerm", NC_DOUBLE, ("TimeHorizon",))
    linear_term[:] = linear_term_pypsa.loc[idx_name] * n.snapshot_weightings.objective.values

    # ConstantTerm
    constant_term = tub.createVariable("ConstantTerm", NC_DOUBLE)
    constant_term[:] = 0.0

    # MinUpTime
    min_up_time = tub.createVariable("MinUpTime", NC_DOUBLE)
    min_up_time[:] = 0.0

    # MinDownTime
    min_down_time = tub.createVariable("MinDownTime", NC_DOUBLE)
    min_down_time[:] = 0.0

    # InitialPower
    initial_power = tub.createVariable("InitialPower", NC_DOUBLE)
    initial_power[:] = 1.

    # InitUpDownTime
    init_up_down_time = tub.createVariable("InitUpDownTime", NC_DOUBLE)
    init_up_down_time[:] = 1.0

    # InertiaCommitment
    inertia_commitment = tub.createVariable("InertiaCommitment", NC_DOUBLE)
    inertia_commitment[:] = 0.0

    id_thermal += 1

### 2.5 Add an IntermittentUnitBlock for each renewable generator

In [16]:
renewable_generators = n.generators[n.generators.index.isin(renewable_carriers)]

id_ren = id_thermal

if not renewable_generators.empty:
    for (idx_name, row) in renewable_generators.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_ren}")
        tiub.id = str(id_ren)
        tiub.type = "IntermittentUnitBlock"

        n_max_power = n.generators_t.p_max_pu.loc[:, idx_name]
        

        # max power
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE, ("TimeHorizon",))
        max_power[:] = n_max_power

        # # max capacity
        # max_capacity = tiub.createVariable("MaxCapacity", NC_DOUBLE)
        # max_capacity[:] = row[1].p_nom_max 

        id_ren += 1

### 2.6 Add a BatteryUnitBlock for each battery

In [17]:
hydro_systems_i = n.storage_units_t.inflow.columns  # index of hydro systems (storage units with inflow)
batteries_i = n.storage_units.index.difference(hydro_systems_i)  # index of batteries (storage units without inflow)

hydro_systems = n.storage_units.loc[hydro_systems_i]
batteries = n.storage_units.loc[batteries_i]

id_batt = id_ren

if not batteries.empty:
    for (idx_name, row) in batteries.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_batt}")
        tiub.id = str(id_batt)
        tiub.type = "BatteryUnitBlock"
        

        # max power
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE)
        max_power[:] = row.p_max_pu
        
        # min power
        min_power = tiub.createVariable("MinPower", NC_DOUBLE)
        min_power[:] = row.p_min_pu

        # max energy
        max_storage = tiub.createVariable("MaxStorage", NC_DOUBLE)
        max_storage[:] = row.max_hours

        # max energy
        min_storage = tiub.createVariable("MinStorage", NC_DOUBLE)
        min_storage[:] = 0.0

        # Initial storage
        initial_storage = tiub.createVariable("InitialStorage", NC_DOUBLE)
        initial_storage[:] = row.state_of_charge_initial * row.max_hours

        # Storing battery efficiency
        storing_efficiency = tiub.createVariable("StoringBatteryRho", NC_DOUBLE)
        storing_efficiency[:] = row.efficiency_store

        # Discharge battery efficiency
        storing_efficiency = tiub.createVariable("ExtractingBatteryRho", NC_DOUBLE)
        storing_efficiency[:] = row.efficiency_dispatch

        # # max capacity
        # max_capacity = tiub.createVariable("MaxCapacity", NC_DOUBLE)
        # max_capacity[:] = row[1].p_nom_max 

        id_batt += 1

### 2.7 Add a SlackUnitBlock to each load curtailment unit

In [18]:
slack_generators = n.generators[n.generators.carrier.isin(slack_carriers)]

id_slack = id_batt

if not slack_generators.empty:
    for (idx_name, row) in slack_generators.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_slack}")
        tiub.id = str(id_slack)
        tiub.type = "SlackUnitBlock"
        

        # max power
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE)
        max_power[:] = row.p_nom_opt
        
        # max power
        active_power_cost = tiub.createVariable("ActivePowerCost", NC_DOUBLE, ("TimeHorizon",))
        active_power_cost[:] = row.marginal_cost * n.snapshot_weightings.objective.values

        # # max capacity
        # max_capacity = tiub.createVariable("MaxCapacity", NC_DOUBLE)
        # max_capacity[:] = row[1].p_nom_max 

        id_slack += 1

### 2.8 Add a HydroUnitBlock for each hydro unit

In [19]:
id_hydro = id_slack

if not hydro_systems.empty:
    for (idx_name, row) in hydro_systems.iterrows():

        tiub = master.createGroup(f"UnitBlock_{id_hydro}")
        tiub.id = str(id_hydro)
        tiub.type = "HydroUnitBlock"

        tiub.createDimension("NumberReservoirs", 1)  # optional, the number of reservoirs
        N_ARCS = 3
        tiub.createDimension("NumberArcs", N_ARCS)  # optional, the number of arcs connecting the reservoirs
        # No NumberIntervals
        
        MAX_FLOW = 100*n.storage_units_t.inflow.loc[:, idx_name].max()
        P_MAX = row.p_max_pu
        P_MIN = row.p_min_pu

        # StartArc
        start_arc = tiub.createVariable("StartArc", NC_UINT, ("NumberArcs",))
        start_arc[:] = np.full((N_ARCS,), 0, dtype=NP_UINT)

        # EndArc
        end_arc = tiub.createVariable("EndArc", NC_UINT, ("NumberArcs",))
        end_arc[:] = np.full((N_ARCS,), 1, dtype=NP_UINT)

        # MaxPower
        max_power = tiub.createVariable("MaxPower", NC_DOUBLE, ("NumberArcs",)) #, ("NumberArcs",)) #, ("TimeHorizon",)) #"NumberArcs"))
        max_power[:] = np.array([P_MAX, 0., 0.], dtype=NP_DOUBLE)

        # MinPower
        min_power = tiub.createVariable("MinPower", NC_DOUBLE, ("NumberArcs",)) #, ("NumberArcs",)) #, ("TimeHorizon",)) #"NumberArcs"))
        min_power[:] = np.array([0., 0., P_MIN], dtype=NP_DOUBLE)

        # MinFlow
        min_flow = tiub.createVariable("MinFlow", NC_DOUBLE, ("NumberArcs",)) #, ("TimeHorizon",))
        min_flow[:] = np.array([0., 0., -MAX_FLOW], dtype=NP_DOUBLE)

        # MaxFlow
        max_flow = tiub.createVariable("MaxFlow", NC_DOUBLE, ("NumberArcs",)) #, ("TimeHorizon",))
        max_flow[:] = np.array([P_MAX * 100., MAX_FLOW, 0.], dtype=NP_DOUBLE)
        
        # MinVolumetric
        min_volumetric = tiub.createVariable("MinVolumetric", NC_DOUBLE) #, ("TimeHorizon",))
        min_volumetric[:] = 0.0

        # MaxVolumetric
        max_volumetric = tiub.createVariable("MaxVolumetric", NC_DOUBLE)
        max_volumetric[:] = row.p_nom_opt * row.max_hours

        
        # Inflows
        inflows = tiub.createVariable("Inflows", NC_DOUBLE, ("NumberReservoirs", "TimeHorizon")) #,"NumberReservoirs",))  #"NumberReservoirs", 
        inflows[:] = np.array([n.storage_units_t.inflow.loc[:, idx_name]])

        # InitialVolumetric
        initial_volumetric = tiub.createVariable("InitialVolumetric", NC_DOUBLE) #, ("NumberReservoirs",))
        initial_volumetric[:] = row.state_of_charge_initial * row.max_hours * row.p_nom_opt

        # NumberPieces
        pieces = np.full((N_ARCS,), 1, dtype=NP_UINT)
        number_pieces = tiub.createVariable("NumberPieces", NC_UINT, ("NumberArcs",))
        number_pieces[:] = pieces

        # TotalNumberPieces
        tiub.createDimension("TotalNumberPieces", pieces.sum())

        # LinearTerm
        linear_term = tiub.createVariable("LinearTerm", NC_DOUBLE, ("TotalNumberPieces",))
        # linear_term[:] = np.array([1/n.storage_units.loc[idx_name, "efficiency_dispatch"], 0., n.storage_units.loc[idx_name, "efficiency_store"]], dtype=NP_DOUBLE)
        linear_term[:] = np.array([1/n.storage_units.loc[idx_name, "efficiency_dispatch"], 0., n.storage_units.loc[idx_name, "efficiency_store"]], dtype=NP_DOUBLE)

        # ConstTerm
        const_term = tiub.createVariable("ConstantTerm", NC_DOUBLE, ("TotalNumberPieces",))
        const_term[:] = np.full((N_ARCS,), 0.0, dtype=NP_DOUBLE)

Finally close the netcdf problem file

In [20]:
ds.close()

## 4. Execute UCBlockSolver

In [22]:
import subprocess
import os

PROJECT_PATH = "../../smspp-project"   # Path of the SMSpp project
COMPILE_MODE = "Release"              # Compilation mode (Debug/Release)

DATA_FOLDER = folder_builder  # Folder where all data inputs are contained (cwd will be moved here)
BSC_NAME = "BSPar.txt"          # Name of the file describing the BlockSolverConfig
IBFILE_NAME = "pypsa2smspp.nc4"      # Name of the UC-file to test
CONFIG_DIR = "./config/"

PARENT_ABSPATH_UCS = os.path.abspath(PROJECT_PATH + "/build/InvestmentBlock/test/" + COMPILE_MODE)
UCS_ABSPATH = os.path.abspath(PARENT_ABSPATH_UCS + "/InvestmentBlock_test.exe")

result = subprocess.run(
    [UCS_ABSPATH, "-c", CONFIG_DIR, "-S", CONFIG_DIR + BSC_NAME, IBFILE_NAME],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    cwd=DATA_FOLDER,
)
result_ascii = result.stdout.decode('ascii')
print(result_ascii)

pypsa2smspp.nc4 is a block file.
Block configuration was not provided. Using default configuration.
Using Solver configuration in ./config/BSPar.txt.
Using Solver configuration in ./config/uc_solverconfig.txt.

{1-0-0-0.0001} t = 1.00e-10 ~ D*_1( z* ) = 0.00e+00 ~ Sigma = 0.00e+00
            Fi undefined
    Lambda1 = [ 0.00e+00 0.00e+00 0.00e+00 ]
    UB[ 0 ] = INF, LB[ 0 ] = -INF
            Fi[ 0 ]: UB = 2.3753582465e+09, LB = 2.3753582465e+09 [0.0136] 
            New subgradient for Fi[ 0 ] ~ Alfa1 = -2.38e+09 ~ gd = -0.00e+00 stored in 0 (0)
            [0.0137] Fi1 = 2.3753582465e+09
            Fi1 defined ==> SS 

{1-1-1-0.0138} t = 1.00e+00 ~ D*_1( z* ) = 1.50e+12 ~ Sigma = 1.21e+14
            Fi = 2.3753582465e+09 ~ eU = 1.14e+05
    Lambda1 = [ 1.00e+06 1.00e+06 1.00e+06 ]
    UB[ 0 ] = INF, LB[ 0 ] = -1.2429680401e+14
            Fi[ 0 ]: UB = 5.6348856768e+08, LB = 5.6348856768e+08 [0.0075] 
            New subgradient for Fi[ 0 ] ~ Alfa1 = 2.38e+09 ~ gd = 5.63e+08 stor

## 4. Validate the objective function of SMS++ and compare it to SMSpp

We read the output of the investment solver and compare it to the objective obtained from PyPSA

In [24]:
res = re.search("Solution value: (.*)\r\n", result_ascii)
smspp_obj = float(res.group(1))
print("SMS++ obj         : %.6f" % smspp_obj)
print("PyPSA dispatch obj: %.6f" % n.objective)
print("Error SMS++ - PyPSA dispatch [%%]: %.5f" % (100*(smspp_obj - n.objective)/n.objective))

SMS++ obj         : 26442.491524
PyPSA dispatch obj: 26442.491211
Error SMS++ - PyPSA dispatch [%]: 0.00000
